In [3]:
from utils.df_handle import *

Default bq project:  spatial-vision-343005.biteam


In [ ]:
import pendulum
from airflow import DAG
from airflow.operators.dummy_operator import DummyOperator
from airflow.operators.python_operator import PythonOperator

local_tz = pendulum.timezone("Asia/Bangkok")

name='PDA_SO'
prefix='SYNC_'
csv_path = '/usr/local/airflow/plugins'+'/'

dag_params = {
    'owner': 'airflow',
    "depends_on_past": False,
    'start_date': datetime(2022, 5, 10, tzinfo=local_tz),
    'email_on_failure': True,
    'email_on_retry': False,
    'email':['duyvq@merapgroup.com', 'vanquangduy10@gmail.com'],
    'do_xcom_push': False,
    'execution_timeout':timedelta(seconds=300)
    # 'retries': 3,
    # 'retry_delay': timedelta(minutes=10),
}

dag = DAG(prefix+name,
          catchup=False,
          default_args=dag_params,
          schedule_interval= '5 * * * *',
          tags=[prefix+name, 'Daily', '60mins']
)

In [4]:
start_date = '2022-01-01'
datenow = datetime.now().strftime("%Y-%m-%d")
datenow_mns1 = (datetime.now() - timedelta(days=1)).strftime("%Y-%m-%d")
fdom = datetime.now().replace(day=1).strftime("%Y-%m-%d")
datenow_mns45 = (datetime.now() - timedelta(days=45)).strftime("%Y-%m-%d")

In [4]:
# datenow_mns45

'2022-03-31'

In [6]:
from_tb = "OM_PDASalesOrd"

In [28]:
sql = \
f"""
DECLARE @from DATE = '{start_date}'
DECLARE @to DATE = '2022-01-31'
SELECT
CONCAT(BranchID,OrderNbr) as pk,
BranchID,
OrderNbr,
CustID,
SlsPerID,
Status,
Crtd_Prog,
Crtd_User,
Crtd_DateTime,
LUpd_DateTime,
LUpd_User,
Remark,
InsertFrom,
BranchRouteID,
SalesRouteID,
DeliveryTime,
OriOrderNbrUp,
Version
from {from_tb}
where Crtd_Datetime >= @from
"""

In [21]:
table_name = "sync_dms_pda_so"
table_temp = "sync_dms_pda_so_temp"

In [22]:
print(sql)


DECLARE @from DATE = '2022-05-01'
DECLARE @to DATE = '2022-01-31'
SELECT
CONCAT(BranchID,OrderNbr) as pk,
BranchID,
OrderNbr,
CustID,
SlsPerID,
Status,
Crtd_Prog,
Crtd_User,
Crtd_DateTime,
LUpd_DateTime,
LUpd_User,
Remark,
InsertFrom,
BranchRouteID,
SalesRouteID,
DeliveryTime,
OriOrderNbrUp,
Version
from OM_PDASalesOrd
where Crtd_Datetime >= @from



In [ ]:
usql = \
f"""
DECLARE @from DATE = '{datenow}'
DECLARE @to DATE = '2022-01-31'
SELECT
CONCAT(BranchID,OrderNbr) as pk,
BranchID,
OrderNbr,
CustID,
SlsPerID,
Status,
Crtd_Prog,
Crtd_User,
Crtd_DateTime,
LUpd_DateTime,
LUpd_User,
Remark,
InsertFrom,
BranchRouteID,
SalesRouteID,
DeliveryTime,
OriOrderNbrUp,
Version
from {from_tb}
where Crtd_Datetime >= @from
"""

In [13]:
# print(sql)

In [23]:
df = get_ms_df(sql)
df['inserted_at'] = datetime.now()

In [25]:
bq_values_insert(df, f"{table_temp}", 3)

1it [00:03,  3.97s/it]


In [26]:
sql = \
f"""
DROP TABLE IF EXISTS biteam.{table_name};
CREATE TABLE biteam.{table_name} LIKE biteam.{table_temp}
PARTITION BY DATE(crtd_datetime)
CLUSTER BY pk,branchid,ordernbr,slsperid
"""

In [27]:
execute_bq_query(sql)

In [15]:
# df = get_ms_df(sql)
# df['inserted_at'] = datetime.now()
# table_name = "sync_dms_pda_so"
# table_temp = "sync_dms_pda_so_temp"
# sql = \
# f"""
# delete from biteam.sync_dms_pda_so where date(crtd_datetime) >= '{datenow}'
# """
# print("delete_sql: ", sql)
# execute_bq_query(sql)
# bq_values_insert(df, f"{table_temp}", 3)
sql = \
f"""
DROP TABLE IF EXISTS biteam.{table_name};
CREATE TABLE biteam.{table_name} LIKE biteam.{table_temp}
PARTITION BY DATE(crtd_datetime)
CLUSTER BY pk,branchid,ordernbr,slsperid
"""

In [29]:
df = get_ms_df(sql)
df['inserted_at'] = datetime.now()
table_name = "sync_dms_pda_so"
table_temp = "sync_dms_pda_so_temp"
bq_values_insert(df, f"{table_name}", 2)

1it [00:13, 13.92s/it]


In [51]:
# execute_bq_query(sql)

In [24]:
# bq_values_insert(df, f"{table_name}", 2)

1it [00:04,  4.79s/it]


In [25]:
usql = \
f"""
DECLARE @from DATE = '{datenow}'
DECLARE @to DATE = '2022-01-31'
SELECT
BranchID,
OrderNbr,
CustID,
SlsPerID,
Status,
Crtd_Prog,
Crtd_User,
Crtd_DateTime,
LUpd_User,
LUpd_DateTime,
Remark,
InsertFrom,
BranchRouteID,
SalesRouteID,
DeliveryTime,
OriOrderNbrUp,
Version
from {from_tb}
where LUpd_DateTime >= @from
and Crtd_DateTime < @from
"""

In [28]:
print(sql)


DECLARE @from DATE = '2022-05-14'
DECLARE @to DATE = '2022-01-31'
SELECT
BranchID,
OrderNbr,
CustID,
SlsPerID,
Status,
Crtd_Prog,
Crtd_User,
Crtd_DateTime,
LUpd_DateTime,
Remark,
InsertFrom,
BranchRouteID,
SalesRouteID,
DeliveryTime,
OriOrderNbrUp,
Version
from OM_PDASalesOrd
where LUpd_DateTime >= @from
and Crtd_DateTime < @from



In [29]:
#UPDATE
df_update = get_ms_df(sql)
df_update.columns = lower_col(df_update)
# df_update.columns
df_update['crtd_datetime'] = df_update.crtd_datetime.dt.normalize()
df_update1 = df_update['crtd_datetime']
df_update1.drop_duplicates(inplace=True)
tpl_dt = tuple(df_update1.dt.strftime('%Y-%m-%d').to_list()) + ('1900-01-01',)
# tpl_dt
df_update1 = df_update['branchid'] + df_update['ordernbr']
df_update1.drop_duplicates(inplace=True)
tpl_pk = tuple(df_update1.to_list()) + ('',)
del_sql = \
f"""
DELETE FROM biteam.sync_dms_pda_so
WHERE
DATE(crtd_datetime) in {tpl_dt}
AND concat(branchid,ordernbr) in {tpl_pk}
"""
print(del_sql)
execute_bq_query(del_sql)
df_update['inserted_at'] = datetime.now()
bq_values_insert(df_update, f"{table_name}", 2)

In [ ]:
#SYNCBACK

df = get_ms_df(syncb_sql)

df['inserted_at'] = datetime.now()

bq_values_insert(df, f"{table_name}", 2)

dsql = \
f"""
delete from biteam.{table_name} where date(crtd_datetime) >= '{datenow_mns45}'
"""
print("delete_sql: ", dsql)

execute_bq_query(dsql)

In [1]:
# 20220628

sql = \
"""
DECLARE @from DATE = '2022-06-28'
DECLARE @to DATE = '2022-01-31'
SELECT
concat(BranchID,OrderNbr) as pk,
BranchID,
OrderNbr,
CustID,
SlsPerID,
Status,
Crtd_Prog,
Crtd_User,
Crtd_DateTime,
LUpd_DateTime,
Lupd_User,
Remark,
InsertFrom,
BranchRouteID,
SalesRouteID,
DeliveryTime,
OriOrderNbrUp,
Version,
OrderType
from OM_PDASalesOrd
where cast(LUpd_DateTime as DATE) >= @from
"""

In [4]:
df = get_ms_df(sql)

In [5]:
df.head()

,pk,BranchID,OrderNbr,CustID,SlsPerID,Status,Crtd_Prog,Crtd_User,Crtd_DateTime,LUpd_DateTime,Lupd_User,Remark,InsertFrom,BranchRouteID,SalesRouteID,DeliveryTime,OriOrderNbrUp,Version,OrderType
0,MR0001DH0-0622-04796,MR0001,DH0-0622-04796,N02208293,MR1406,C,SFA,MR1406,2022-06-28 08:26:37,2022-06-28 08:31:45,admin,,S,RHCM01,TMD2_4,24,,42918,IN
1,MR0001DH0-0622-04797,MR0001,DH0-0622-04797,001761,MR2556,C,SFA,MR2556,2022-06-28 08:26:58,2022-06-28 08:31:46,admin,,S,RHCM01,TMD1_1,24,,51377,IN
2,MR0001DH0-0622-04798,MR0001,DH0-0622-04798,M2301021,MR1995,C,SFA,MR1995,2022-06-28 08:28:53,2022-06-28 08:31:46,admin,,S,RHCM01,MCP-NT-HCM6,24,,61859,IN
3,MR0001DH0-0622-04799,MR0001,DH0-0622-04799,P4708-0151,MR1179,C,SFA,MR1179,2022-06-28 08:35:50,2022-06-28 08:41:54,admin,,S,RHCM01,MCP-DL-HCM2,24,,18183,IN
4,MR0001DH0-0622-04800,MR0001,DH0-0622-04800,M1001010,MR1363,C,OM10101,MR1363,2022-06-28 08:36:24,2022-06-28 08:38:22,MR1363,,O,,,24,,70157,IN
